In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import string
from tqdm import tqdm

tqdm.pandas(desc='progress-bar')

In [2]:
data_dir = '../../data/'
train = pd.read_csv(data_dir + 'train.csv', index_col = 'id')
train.head()

,qid1,qid2,question1,question2,is_duplicate
id,,,,,
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
def process(x):
    import nltk
    import string 
    from nltk import word_tokenize
    from nltk.corpus import stopwords
    from nltk.stem.wordnet import WordNetLemmatizer
    
    if x:
        stop_words = stopwords.words('english') + list(string.punctuation)

        tokens = word_tokenize(x.lower())
        updated = list(set(tokens) - set(stop_words))

        lemma = WordNetLemmatizer()

        lemmatized = [lemma.lemmatize(word) for word in updated]

        return lemmatized
    else:
        return []

In [4]:
def create_synonym(row):
    proc1 = process(row.question1)
    proc2 = process(row.question2)
    
    if len(proc1) > len(proc2):
        larger =  proc1 
        smaller = proc2
    else:
        larger = proc2
        smaller = proc1
        
    num_syns = 0
    
    for word1 in larger:
        for word2 in smaller:
            for i,j in enumerate(wordnet.synsets(word1)):
                if word2 in j.lemma_names():
                    num_syns += 1
                    break
                    
    return num_syns / len(larger)

In [5]:
isnot = train[train.is_duplicate == 0]

In [6]:
itis = train[train.is_duplicate == 1]

In [7]:
itis.iloc[4].question1

'What was your first sexual experience like?'

In [8]:
process(itis.iloc[4].question1)

['experience', 'like', 'first', 'sexual']

In [9]:
itis.iloc[4].question2

'What was your first sexual experience?'

In [10]:
process(itis.iloc[4].question2)

['experience', 'first', 'sexual']

In [11]:
isnot.iloc[4].question1

'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?'

In [12]:
process(isnot.iloc[4].question1)

['quikly',
 'dissolve',
 'sugar',
 'water',
 'salt',
 'oxide',
 'di',
 'methane',
 'one',
 'carbon']

In [13]:
isnot.iloc[4].question2

'Which fish would survive in salt water?'

In [14]:
process(isnot.iloc[4].question2)

['water', 'survive', 'fish', 'salt', 'would']

In [15]:
create_synonym(isnot.iloc[4])

0.2

In [16]:
create_synonym(itis.iloc[4])

0.75

In [18]:
itis.iloc[5].question1

'What would a Trump presidency mean for current international master’s students on an F1 visa?'

In [20]:
itis.iloc[5].question2

'How will a Trump presidency affect the students presently in US or planning to study in US?'

In [22]:
create_synonym(itis.iloc[5])

0.2727272727272727

In [23]:
process(itis.iloc[5].question1)

['would',
 'student',
 '’',
 'f1',
 'master',
 'presidency',
 'mean',
 'international',
 'trump',
 'current',
 'visa']

In [24]:
process(itis.iloc[5].question2)

['presently',
 'planning',
 'affect',
 'study',
 'student',
 'u',
 'presidency',
 'trump']

In [26]:
def create_unique(row):
    proc1 = process(row.question1)
    proc2 = process(row.question2)
    one_bigger_two = set(proc1) - set(proc2)
    two_bigger_one = set(proc2) - set(proc1)
    
    return len(one_bigger_two) + len(two_bigger_one)

In [28]:
create_unique(itis.iloc[5])

13